Pandas is actually set of software components , much of  which is not even written in Python. <br>

### REST API

Rest API’s function by sending a request, the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

In cryptocurrency a popular method to display the movements of the price of a currency.




![Example Candlestick Graph](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/candlestick.png)

Here is a description of the candle sticks.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/explanation.png" alt="Candle Stick Definition" width="800" height="600">

In this lab, we will be using the <a href=https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork1005-2022-01-01>CoinGecko API</a> to create one of these candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called <a href=https://github.com/man-c/pycoingecko>PyCoinGecko</a>. **PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.**


In [ ]:
# !pip install pycoingecko
# !pip install plotly
# !pip install mplfinance
# !pip install --upgrade nbformat

In [13]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

In [14]:
cg = CoinGeckoAPI()

ethereum_data = cg.get_coin_market_chart_by_id(id='ethereum', vs_currency='inr', days=60)

In [15]:
type(ethereum_data)

dict

In [16]:
df = pd.DataFrame(ethereum_data)
df

,prices,market_caps,total_volumes
0,"[1668524483841, 102420.71772382625]","[1668524483841, 12323285839320.287]","[1668524483841, 735494564046.3485]"
1,"[1668528090466, 102790.55365669787]","[1668528090466, 12410652000034.217]","[1668528090466, 732617925771.6057]"
2,"[1668531711019, 102529.17603530029]","[1668531711019, 12360286412158.648]","[1668531711019, 729740502797.1875]"
3,"[1668535230176, 102798.44749484539]","[1668535230176, 12416879650339.338]","[1668535230176, 729156512311.9954]"
4,"[1668538839298, 100704.99678576384]","[1668538839298, 12126711688550.246]","[1668538839298, 750493346331.7883]"
...,...,...,...
1442,"[1673694088442, 123465.24573116387]","[1673694088442, 14871304896081.818]","[1673694088442, 1276509166762.4307]"
1443,"[1673697725519, 124582.34335219298]","[1673697725519, 15011458873713.12]","[1673697725519, 1293616744114.145]"
1444,"[1673701222132, 125268.43969876593]","[1673701222132, 15091543585510.273]","[1673701222132, 1323539803170.4434]"
1445,"[1673704823522, 124712.7734498486]","[1673704823522, 15006626637648.053]","[1673704823522, 1329303341734.8596]"


In [17]:
ethereum_price_data = ethereum_data['prices']
ethereum_price_data[0:5]

[[1668524483841, 102420.71772382625],
 [1668528090466, 102790.55365669787],
 [1668531711019, 102529.17603530029],
 [1668535230176, 102798.44749484539],
 [1668538839298, 100704.99678576384]]

In [18]:
data = pd.DataFrame(ethereum_price_data, columns=['TimeStamp', 'Price'])
data

,TimeStamp,Price
0,1668524483841,102420.717724
1,1668528090466,102790.553657
2,1668531711019,102529.176035
3,1668535230176,102798.447495
4,1668538839298,100704.996786
...,...,...
1442,1673694088442,123465.245731
1443,1673697725519,124582.343352
1444,1673701222132,125268.439699
1445,1673704823522,124712.773450


In [19]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))
data

,TimeStamp,Price,date
0,1668524483841,102420.717724,2022-11-15
1,1668528090466,102790.553657,2022-11-15
2,1668531711019,102529.176035,2022-11-15
3,1668535230176,102798.447495,2022-11-15
4,1668538839298,100704.996786,2022-11-16
...,...,...,...
1442,1673694088442,123465.245731,2023-01-14
1443,1673697725519,124582.343352,2023-01-14
1444,1673701222132,125268.439699,2023-01-14
1445,1673704823522,124712.773450,2023-01-14


In [20]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
candlestick_data

date          Price                                             
                          min            max          first           last
0   2022-11-15  102420.717724  102798.447495  102420.717724  102798.447495
1   2022-11-16   97642.147445  103132.199467  100704.996786   99073.098996
2   2022-11-17   97290.344421   99613.452257   98741.888016   98689.812992
3   2022-11-18   97976.217314  100124.245001   98942.894284   98420.377060
4   2022-11-19   98264.894205   98924.340546   98378.331797   98625.935122
..         ...            ...            ...            ...            ...
56  2023-01-10  108356.848925  109705.807124  109705.807124  108792.760824
57  2023-01-11  108104.476549  109543.606219  109524.051458  108456.608280
58  2023-01-12  109087.228771  116131.404903  109087.228771  116131.404903
59  2023-01-13  114309.989501  115905.538578  115579.980252  115398.706179
60  2023-01-14  115360.408534  127138.649959  115421.055357  124335.703980

[61 rows x 5 columns]

In [21]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])
                ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()